In [1]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import numpy as np
from math import *
import scipy.stats
from pandas import *
from scipy import linalg
#import scipy.integrate as integrate
from scipy.integrate import quad
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from mpl_toolkits import mplot3d

import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.graph_objects as go

from tqdm.notebook import tqdm

In [2]:
# Define function to calculate jump distribution for integral terms

def jump_distribution_full(x_value,mu,sigma_jump,step_x):
    lower = x_value - step_x/2.0
    upper = x_value + step_x/2.0
    #print(lower,upper)
    def normal_distribution_function(x):
        value = scipy.stats.norm.pdf(x,mu,sigma_jump)
        if sigma_jump == 0.0:
            return 0.0
        else:
            return value
    res, err = quad(normal_distribution_function, lower, upper)
    return res  #output is a vector of the average densities around the regions of each value in the vector x

In [3]:
# Coefficients as obtained from Alternating Direction Implicit (ADI) discretization scheme

def a_coeff_sv(x_value, y_value, ksi, kappa, theta, theta_vol, omega):
    
    term_1 = np.absolute(kappa*(theta-x_value)/dx)
    term_2 = (y_value/(dx**2)) + ((ksi**2)*y_value/(dy**2)) # will become 0 when y_value = y0 = 0
    term_3 = np.absolute(theta_vol*(omega-y_value)/dy)*(y_value != y[-1]) # will become theta_vol*omega when y_value = y0 = 0
    
    coefficient = term_1 + term_2 + term_3
    return coefficient

def b_coeff_sv(x_value, y_value, ksi, kappa, theta, theta_vol, omega):
    
    term_1 = (kappa*(theta-x_value)/dx)*(kappa*(theta-x_value) > 0)
    term_2 = (y_value/(2*(dx**2))) # will become 0 when y_value = y0 = 0
    
    coefficient_2 = term_1 + term_2
    return coefficient_2

def c_coeff_sv(x_value, y_value, ksi, kappa, theta, theta_vol, omega):
    
    term_1 = -(kappa*(theta-x_value)/dx)*(kappa*(theta-x_value) < 0)
    term_2 = (y_value/(2*(dx**2))) # will become 0 when y_value = y0 = 0 
    
    coefficient_3 = term_1 + term_2
    return coefficient_3 

def d_coeff_sv(y_value, ksi, kappa, theta, theta_vol, omega):
    
    term_1 = ((ksi**2)*y_value/(2*(dy**2)))*(y_value != y[-1]) # will become 0 when y_value = y0 = 0 
    term_2 = (theta_vol*(omega-y_value)/dy)*(theta_vol*(omega-y_value) > 0)*(y_value != y[-1])
    
    coefficient_4 = term_1 + term_2
    return coefficient_4

def e_coeff_sv(y_value, ksi, kappa, theta, theta_vol, omega):
    
    term_1 = ((ksi**2)*y_value/(2*(dy**2)))*(y_value != y[-1]) # will become 0 when y_value = y0 = 0 
    term_2 = -(theta_vol*(omega-y_value)/dy)*(theta_vol*(omega-y_value) < 0)*(y_value != y[-1])*(y_value != y[0])
    term_3 = ((ksi**2)*y_value/(dy**2))*(y_value == y[-1])
    
    coefficient_5 = term_1 + term_2 + term_3
    return coefficient_5

In [4]:
# Construction of M matrix containing the coefficients of the solution at time t = q+1 in the implicit scheme for a given volatility value (block)

def M_matrix_sv(x,sigma,ksi, kappa, theta, theta_vol, omega, L):  
    M = np.zeros(shape=(len(x),len(x)))
    for i in range(0,len(x)):
        if x[i] <= 0.0 or x[i] >= L:
            M[i,i] = 1.0
        else:
            a_coefficient = a_coeff_sv(x[i], sigma, ksi, kappa, theta, theta_vol, omega)
            b_coefficient = b_coeff_sv(x[i], sigma, ksi, kappa, theta, theta_vol, omega)
            c_coefficient = c_coeff_sv(x[i], sigma, ksi, kappa, theta, theta_vol, omega)
            if a_coefficient < 0 or b_coefficient < 0 or c_coefficient <0:
                print('Error')
            #print(x[(i-1):(i+2)])
            M[i,(i-1):(i+2)] = [-dt*c_coefficient, 1+dt*a_coefficient, -dt*b_coefficient]

    return M

In [5]:
# Construction of full block M matrix containing the coefficients of the solution at time t = q+1 across discretized volatility grid

def M_final_sv(x,y,ksi, kappa, theta, theta_vol, omega, L_vec):
    M_final_matrix = np.zeros(shape=(1,len(x)*len(y)))
    #M_first = M_matrix_sv(x,y[0], 0, kappa,theta,jump_rate)
    #M_last = M_matrix_sv(x, y[len(y)-1], kappa, theta, jump_rate)
    
    for i in tqdm(range(0,len(y))):
        #if i == 0 or i == len(y)-1:
        #    M_block = M_matrix_sv(x,y[i], 0, kappa, theta, L_vec[i])
        #else:
        M_block = M_matrix_sv(x, y[i], ksi, kappa, theta, theta_vol, omega, L_vec[i])
        for j in range(0,len(y)):
            if j < i:
                e_c = e_coeff_sv(y[i], ksi, kappa, theta, theta_vol, omega)
                X_current = np.eye(len(x))*(-dt*e_c*(j == (i-1)))
                X_current[x <= 0.0] = 0.0
                X_current[x >= L_vec[i]] = 0.0
                M_block = np.block([X_current, M_block])
                #print(f' Vol {y[i]} to {y[j]} --- e {e_c*(j == (i-1))}')
            if j > i:
                d_c = d_coeff_sv(y[i], ksi, kappa, theta, theta_vol, omega)
                X_current = np.eye(len(x))*(-dt*d_c*(j == (i+1)))
                X_current[x <= 0.0] = 0.0
                X_current[x >= L_vec[i]] = 0.0
                M_block = np.block([ M_block, X_current]) 
                #print(f' Vol {y[i]} to {y[j]} --- d {d_c*(j == (i+1))}')
            
            if (e_coeff_sv(y[i], ksi, kappa, theta, theta_vol, omega) < 0 or d_coeff_sv(y[i], ksi, kappa, theta, theta_vol, omega) < 0 ):
                print('Error - sv coeffs')
        
        M_final_matrix = np.block([
            [M_final_matrix],
            [M_block]
        ])
        
    #M_final_sv_matrix = M_final_matrix[1:,:]
    #M_final_sv_matrix[0:len(x),0:len(x)] = M_first
    #M_final_matrix[(len(y)-1)*len(x):len(y)*len(x),(len(y)-1)*len(x):len(y)*len(x)] = M_last
    
    
    
    return M_final_matrix[1:,:]

In [6]:
# Calculation of integral term in the PIDE

def jump_integral(x, mu, N, sigma_jump, step_x):
    integral_list = []
    positions = np.arange(int(-N/2+1),int(N/2),1)
    for j in positions: 
        x_jump = step_x*j
        integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
        #print(x_jump, integral)
        integral_list.append(integral)

    return sum(integral_list)

In [7]:
# Construction of matrix containing the coefficients of the solution at time t = q in the implicit scheme for a given volatility value y

def N_matrix_sv(x, poisson_rate, N, mu, sigma_jump, step_x, L):
    N_m = np.zeros(shape=(len(x),len(x)))
    positions = np.arange(int(-N/2+1),int(N/2),1)
    jump_result_list = []
    for i in range(0,(len(x))):
        integral_list = []
        if x[i] > 0.0 and x[i] < L:
            for j in positions: 
                x_jump = step_x*j
                if (i+j) >= 0 and (i+j) <= (len(x)-1):
                    integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
                    integral_list.append(integral)
                    N_m[i,i+j] = poisson_rate*dt*integral
                    
            jump_integral = sum(integral_list)
            N_m[i,i] = N_m[i,i]+ (1 - poisson_rate*dt*jump_integral)
            
            jump_result_list.append(jump_integral)
            if 1-poisson_rate*dt*jump_integral < 0:
                print('Error in integral approximation')
                
    print(f'Min: {min(jump_result_list,  default="EMPTY")} Max: {max(jump_result_list,  default="EMPTY")}')
            
    return N_m


In [8]:
# Construction of full block N matrix containing the coefficients of the solution at time t = q+1 across discretized volatility grid

def N_final_sv(x, poisson_rate, N, mu, sigma_jump, step_x, y, L_vec):
    N_final_matrix = np.zeros(shape=(1,len(x)*len(y)))
    for i in tqdm(range(0,len(y))):
        N_block = N_matrix_sv(x, poisson_rate, N, mu, sigma_jump, step_x, L_vec[i])
        for j in range(0,len(y)):
            X_current = np.zeros((len(x),len(x)))
            if j < i:
                N_block = np.block([X_current, N_block])
            if j > i:
                N_block = np.block([N_block, X_current]) 
    
        N_final_matrix = np.block([
            [N_final_matrix],
            [N_block]
        ])
    
    return N_final_matrix[1:,:]

In [9]:
# Construction of boundary condition block matrix for the entire volatility grid

def b_matrix(L_vec, y):
    b_final_matrix = np.array([0])
    for i in range(0,len(y)):
        b_block = np.zeros(len(x))
        b_block[x >= L_vec[i]] = 1.0
        b_final_matrix = np.block([
            b_final_matrix, b_block
        ])
    
    return b_final_matrix[1:]

In [10]:
# Construction of initial condition block matrix for the entire volatility grid 

def phi_matrix_def(x,y,t):
    phi_final_matrix = np.zeros(shape=(1,len(t)))
    for i in range(0, len(y)):
        phi_block = np.zeros(shape = (len(x),len(t)))
        phi_block[:,0] = 1.0 * (x>0.0)
        phi_block[len(x)-1,:] = 1.0
        
        phi_final_matrix = np.block([
            [phi_final_matrix],
            [phi_block]
        ])
        
    return phi_final_matrix[1:,:]

In [11]:
# Define space, time and volatility grid 

x0 = -5.0
xn = 75.0
xsteps = 200
dx = (xn-x0)/xsteps
x = np.arange(x0, xn+dx, dx) #last x value is x = 99 
#print(dx)

t0 = 0.0
tn = 1.0
tsteps = 1000
dt = (tn-t0)/tsteps
t = np.arange(t0, tn+dt, dt) #last time value is t = 999.9
r = dt / (dx**2) # ensure r < 1


y0 = 0.0  # befor yn = 100, ysteps = 50
yn = 200.0
ysteps = 200
dy = (yn-y0)/ysteps
y = np.arange(y0, yn+dy, dy) #last x value is x = 99 

In [ ]:
# Run ADI FD iteration scheme 

L_vec = np.ones(shape=(len(y)))*x[len(x)-1]

kappa_choice = 2.0
theta_choice = 2.0
ksi_choice = 0.07
theta_vol_choice = 0.05
omega_choice = 0.1 #1.0
poisson_rate = 1.0
N_choice = 90
mu_choice = 0.3
sigma_jump_choice = 0.5

M = M_final_sv(x,y,ksi_choice, kappa_choice, theta_choice, theta_vol_choice, omega_choice, L_vec)
N_mat = N_final_sv(x, poisson_rate, N_choice, mu_choice, sigma_jump_choice, dx, y, L_vec)
M_inv = linalg.inv(M)

b = b_matrix(L_vec, y)
phi_matrix = phi_matrix_def(x,y,t)

if y[0] < np.max([(theta_choice*kappa_choice*dx), omega_choice*theta_vol_choice*dy/(ksi_choice**2)]):
    print('Stability conditions may not be satisfied')


for time in tqdm(range(1, len(t))): 
    phi_matrix[:,time] = M_inv.dot(N_mat.dot(phi_matrix[:,time-1]) + b)


In [12]:
# The scheme is computationally expensive - run once, save file and load to analyze 

# phi_matrix = read_csv('phi_matrix_sv_after_comparison.csv', index_col=0)
# phi_matrix = np.array(phi_matrix)
# L = 20.0

In [13]:
xs = np.arange(0,len(x))
phi_average = np.zeros(shape=(len(x),len(t)))
for i in range(0,len(y)):
    phi_average = phi_matrix[xs+i*len(x),] + phi_average
    phi_average_final = phi_average/len(y)
pd_phi_average_final = DataFrame(phi_average_final)

In [13]:
def survival(space,time):
    return phi_matrix[space, time]


def survival_av(space,time):
    return phi_average_final[space,time]

In [14]:
accept_x = np.where((x>=0) & (x<=L))[0]

In [15]:
accept_x_with_zeros = np.append(accept_x, accept_x[-1]+1)

In [16]:
accept_x_with_zeros = np.insert(accept_x_with_zeros,0,accept_x[0]-1)
accept_x_with_zeros_new = np.array([i +2*len(x) for i in accept_x_with_zeros])
accept_x_with_zeros_new

array([414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465],
      dtype=int64)

In [20]:
X = accept_x_with_zeros_new #np.arange(len(x),2*len(x),1)
T = np.arange(0,len(t),1)
X, T = np.meshgrid(X, T)
Phi = survival(X,T)

data=go.Surface(z=Phi, x=x0+dx*accept_x_with_zeros, y=t0+dt*T)

layout = go.Layout(scene = dict(
                    xaxis_title='Initial Position',
                    yaxis_title='Time until maturity',
                    zaxis_title='Survival Probability'))

fig = go.Figure(data=[data], layout=layout)
pyoff.plot(fig)

'temp-plot.html'